In [327]:
import itertools
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import graphviz

from IPython.display import Image
from IPython.core.display import HTML
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
layers = keras.layers

print("Importing done")

Importing done


In [328]:
URL = "https://github.com/Brunel-Visualization/Brunel/raw/master/python/examples/data/whiskey.csv"
path = tf.keras.utils.get_file(URL.split('/')[-1], URL)

print("Data fetching done")

Data fetching done


In [329]:
# Convert the data to a Pandas data frame
data = pd.read_csv(path)

# Shuffle the data
data = data.sample(frac=1)

features = list(data.columns.values)
print(features)

print("Data reading done")

['Name', 'Rating', 'Country', 'Category', 'Price', 'ABV', 'Age', 'Brand']
Data reading done


In [330]:
# Do some preprocessing to limit the # of wine varities in the dataset
data = data[pd.notnull(data['Name'])]
data = data[pd.notnull(data['Price'])]
data = data[pd.notnull(data['Rating'])]

'''
variety_threshold = 10 # Anything that occurs less than this will be removed.
value_counts = data['Brand'].value_counts()
to_remove = value_counts[value_counts <= variety_threshold].index
data.replace(to_remove, np.nan, inplace=True)
data = data[pd.notnull(data['Brand'])]
'''

print("Data processing done")

Data processing done


In [331]:
# Split data into train and test
train_size = int(len(data) * .8)
print ("Total size: %d" % (len(data)))
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

print("Data splitting done")

Total size: 270
Train size: 216
Test size: 54
Data splitting done


In [332]:
# Train features
category_train = data['Category'][:train_size]
name_train = data['Name'][:train_size]

# Train labels
labels_trainOne = data['Price'][:train_size]
labels_trainTwo = data['Rating'][:train_size]

# Test features
category_test = data['Category'][:train_size]
name_test = data['Name'][:train_size]

# Test labels
labels_testOne = data['Price'][:train_size]
labels_testTwo = data['Rating'][:train_size]

print("Labeling done")

Labeling done


In [333]:
# Create a tokenizer to preprocess our text descriptions
vocab_size = 10 # This is a hyperparameter, experiment with different values for your dataset
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(name_train) # only fit on train

print("Tokeninzing done")

Tokeninzing done


In [334]:
# Wide feature 1: sparse bag of words (bow) vocab_size vector
name_bow_train = tokenize.texts_to_matrix(name_train)
name_bow_test = tokenize.texts_to_matrix(name_test)

In [335]:
# Wide feature 2: one-hot vector of variety categories

# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(category_train)
category_train = encoder.transform(category_train)
category_test = encoder.transform(category_test)
num_classes = np.max(category_train) + 1

# Convert labels to one hot
category_train = keras.utils.to_categorical(category_train, num_classes)
category_test = keras.utils.to_categorical(category_test, num_classes)

In [336]:
# Define our wide model with the functional API
bow_inputs = layers.Input(shape=(vocab_size,))
variety_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, variety_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, variety_inputs], outputs=predictions)
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
plot_model(wide_model, to_file='model_plotA.png', show_shapes=True, show_layer_names=True)

Image(url = "model_plotA.png", width = 500, height = 500)

In [337]:
# Deep model feature: word embeddings of wine descriptions
train_embed = tokenize.texts_to_sequences(name_train)
test_embed = tokenize.texts_to_sequences(name_test)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")

In [338]:
# Define our deep model with the Functional API
deep_inputs = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = keras.Model(inputs=deep_inputs, outputs=embed_out)
deep_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
plot_model(deep_model, to_file='model_plotB.png', show_shapes=True, show_layer_names=True)

Image(url = "model_plotB.png", width = 300, height = 300)

In [339]:
# Combine wide and deep into one model
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_model = keras.Model(wide_model.input + [deep_model.input], merged_out)
plot_model(combined_model, to_file='model_plotC.png', show_shapes=True, show_layer_names=True)
combined_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

Image(url = "model_plotC.png", width = 800, height = 800)

In [340]:
# Run training
combined_model.fit([name_bow_train, category_train] + [train_embed], labels_trainOne, epochs=5000, batch_size=512)

Epoch 1/2500
216/216 [==============================] - 2s 9ms/step - loss: 13575.8477 - acc: 0.0000e+00
Epoch 2/2500
216/216 [==============================] - 0s 78us/step - loss: 13567.8516 - acc: 0.0000e+00
Epoch 3/2500
216/216 [==============================] - 0s 71us/step - loss: 13558.6621 - acc: 0.0000e+00
Epoch 4/2500
216/216 [==============================] - 0s 66us/step - loss: 13548.9316 - acc: 0.0000e+00
Epoch 5/2500
216/216 [==============================] - 0s 64us/step - loss: 13538.8428 - acc: 0.0000e+00
Epoch 6/2500
216/216 [==============================] - 0s 29us/step - loss: 13528.4238 - acc: 0.0000e+00
Epoch 7/2500
216/216 [==============================] - 0s 89us/step - loss: 13517.6445 - acc: 0.0000e+00
Epoch 8/2500
216/216 [==============================] - 0s 59us/step - loss: 13506.4561 - acc: 0.0000e+00
Epoch 9/2500
216/216 [==============================] - 0s 44us/step - loss: 13494.6689 - acc: 0.0000e+00
Epoch 10/2500
216/216 [========================

Epoch 78/2500
216/216 [==============================] - 0s 133us/step - loss: 10744.7500 - acc: 0.0000e+00
Epoch 79/2500
216/216 [==============================] - 0s 64us/step - loss: 10676.9980 - acc: 0.0000e+00
Epoch 80/2500
216/216 [==============================] - 0s 115us/step - loss: 10608.9688 - acc: 0.0000e+00
Epoch 81/2500
216/216 [==============================] - 0s 169us/step - loss: 10541.1143 - acc: 0.0000e+00
Epoch 82/2500
216/216 [==============================] - 0s 104us/step - loss: 10472.6699 - acc: 0.0000e+00
Epoch 83/2500
216/216 [==============================] - 0s 103us/step - loss: 10404.0791 - acc: 0.0000e+00
Epoch 84/2500
216/216 [==============================] - 0s 85us/step - loss: 10335.7871 - acc: 0.0000e+00
Epoch 85/2500
216/216 [==============================] - 0s 60us/step - loss: 10267.0586 - acc: 0.0000e+00
Epoch 86/2500
216/216 [==============================] - 0s 105us/step - loss: 10198.3379 - acc: 0.0000e+00
Epoch 87/2500
216/216 [========

Epoch 155/2500
216/216 [==============================] - 0s 150us/step - loss: 7484.4526 - acc: 0.0000e+00
Epoch 156/2500
216/216 [==============================] - 0s 135us/step - loss: 7477.3472 - acc: 0.0000e+00
Epoch 157/2500
216/216 [==============================] - 0s 190us/step - loss: 7470.5415 - acc: 0.0000e+00
Epoch 158/2500
216/216 [==============================] - 0s 71us/step - loss: 7464.0059 - acc: 0.0000e+00
Epoch 159/2500
216/216 [==============================] - 0s 43us/step - loss: 7457.7134 - acc: 0.0000e+00
Epoch 160/2500
216/216 [==============================] - 0s 120us/step - loss: 7451.6411 - acc: 0.0000e+00
Epoch 161/2500
216/216 [==============================] - 0s 116us/step - loss: 7445.7676 - acc: 0.0000e+00
Epoch 162/2500
216/216 [==============================] - 0s 105us/step - loss: 7440.0688 - acc: 0.0000e+00
Epoch 163/2500
216/216 [==============================] - 0s 208us/step - loss: 7434.5249 - acc: 0.0000e+00
Epoch 164/2500
216/216 [======

Epoch 232/2500
216/216 [==============================] - 0s 51us/step - loss: 7137.3345 - acc: 0.0000e+00
Epoch 233/2500
216/216 [==============================] - 0s 135us/step - loss: 7133.7344 - acc: 0.0000e+00
Epoch 234/2500
216/216 [==============================] - 0s 96us/step - loss: 7130.1587 - acc: 0.0000e+00
Epoch 235/2500
216/216 [==============================] - 0s 120us/step - loss: 7126.6030 - acc: 0.0000e+00
Epoch 236/2500
216/216 [==============================] - 0s 57us/step - loss: 7123.0708 - acc: 0.0000e+00
Epoch 237/2500
216/216 [==============================] - 0s 115us/step - loss: 7119.5635 - acc: 0.0000e+00
Epoch 238/2500
216/216 [==============================] - 0s 83us/step - loss: 7116.0811 - acc: 0.0000e+00
Epoch 239/2500
216/216 [==============================] - 0s 64us/step - loss: 7112.6211 - acc: 0.0000e+00
Epoch 240/2500
216/216 [==============================] - 0s 129us/step - loss: 7109.1904 - acc: 0.0000e+00
Epoch 241/2500
216/216 [=========

Epoch 309/2500
216/216 [==============================] - 0s 138us/step - loss: 6935.1426 - acc: 0.0000e+00
Epoch 310/2500
216/216 [==============================] - 0s 154us/step - loss: 6933.4209 - acc: 0.0000e+00
Epoch 311/2500
216/216 [==============================] - 0s 58us/step - loss: 6931.7246 - acc: 0.0000e+00
Epoch 312/2500
216/216 [==============================] - 0s 56us/step - loss: 6930.0493 - acc: 0.0000e+00
Epoch 313/2500
216/216 [==============================] - 0s 58us/step - loss: 6928.3896 - acc: 0.0000e+00
Epoch 314/2500
216/216 [==============================] - 0s 104us/step - loss: 6926.7461 - acc: 0.0000e+00
Epoch 315/2500
216/216 [==============================] - 0s 116us/step - loss: 6925.1245 - acc: 0.0000e+00
Epoch 316/2500
216/216 [==============================] - 0s 40us/step - loss: 6923.5210 - acc: 0.0000e+00
Epoch 317/2500
216/216 [==============================] - 0s 76us/step - loss: 6921.9414 - acc: 0.0000e+00
Epoch 318/2500
216/216 [=========

...

Epoch 2142/2500
216/216 [==============================] - 0s 113us/step - loss: 3272.9106 - acc: 0.0000e+00
Epoch 2143/2500
216/216 [==============================] - 0s 75us/step - loss: 3272.4543 - acc: 0.0000e+00
Epoch 2144/2500
216/216 [==============================] - 0s 93us/step - loss: 3271.9978 - acc: 0.0000e+00
Epoch 2145/2500
216/216 [==============================] - 0s 77us/step - loss: 3271.5532 - acc: 0.0000e+00
Epoch 2146/2500
216/216 [==============================] - 0s 125us/step - loss: 3271.1106 - acc: 0.0000e+00
Epoch 2147/2500
216/216 [==============================] - 0s 170us/step - loss: 3270.6653 - acc: 0.0000e+00
Epoch 2148/2500
216/216 [==============================] - 0s 94us/step - loss: 3270.2388 - acc: 0.0000e+00
Epoch 2149/2500
216/216 [==============================] - 0s 102us/step - loss: 3269.7871 - acc: 0.0000e+00
Epoch 2150/2500
216/216 [==============================] - 0s 88us/step - loss: 3269.3528 - acc: 0.0000e+00
Epoch 2151/2500
216/216 

Epoch 2218/2500
216/216 [==============================] - 0s 102us/step - loss: 3243.5544 - acc: 0.0000e+00
Epoch 2219/2500
216/216 [==============================] - 0s 119us/step - loss: 3243.2222 - acc: 0.0000e+00
Epoch 2220/2500
216/216 [==============================] - 0s 135us/step - loss: 3242.8848 - acc: 0.0000e+00
Epoch 2221/2500
216/216 [==============================] - 0s 189us/step - loss: 3242.5488 - acc: 0.0000e+00
Epoch 2222/2500
216/216 [==============================] - 0s 109us/step - loss: 3242.2173 - acc: 0.0000e+00
Epoch 2223/2500
216/216 [==============================] - 0s 76us/step - loss: 3241.8877 - acc: 0.0000e+00
Epoch 2224/2500
216/216 [==============================] - 0s 193us/step - loss: 3241.5559 - acc: 0.0000e+00
Epoch 2225/2500
216/216 [==============================] - 0s 55us/step - loss: 3241.2183 - acc: 0.0000e+00
Epoch 2226/2500
216/216 [==============================] - 0s 72us/step - loss: 3240.8965 - acc: 0.0000e+00
Epoch 2227/2500
216/21

Epoch 2294/2500
216/216 [==============================] - 0s 99us/step - loss: 3221.4775 - acc: 0.0000e+00
Epoch 2295/2500
216/216 [==============================] - 0s 89us/step - loss: 3221.2266 - acc: 0.0000e+00
Epoch 2296/2500
216/216 [==============================] - 0s 70us/step - loss: 3220.9780 - acc: 0.0000e+00
Epoch 2297/2500
216/216 [==============================] - 0s 88us/step - loss: 3220.7261 - acc: 0.0000e+00
Epoch 2298/2500
216/216 [==============================] - 0s 61us/step - loss: 3220.4756 - acc: 0.0000e+00
Epoch 2299/2500
216/216 [==============================] - 0s 137us/step - loss: 3220.2268 - acc: 0.0000e+00
Epoch 2300/2500
216/216 [==============================] - 0s 64us/step - loss: 3219.9814 - acc: 0.0000e+00
Epoch 2301/2500
216/216 [==============================] - 0s 90us/step - loss: 3219.7358 - acc: 0.0000e+00
Epoch 2302/2500
216/216 [==============================] - 0s 61us/step - loss: 3219.4956 - acc: 0.0000e+00
Epoch 2303/2500
216/216 [==

Epoch 2370/2500
216/216 [==============================] - 0s 55us/step - loss: 3205.1985 - acc: 0.0000e+00
Epoch 2371/2500
216/216 [==============================] - 0s 67us/step - loss: 3205.0269 - acc: 0.0000e+00
Epoch 2372/2500
216/216 [==============================] - 0s 60us/step - loss: 3204.8403 - acc: 0.0000e+00
Epoch 2373/2500
216/216 [==============================] - 0s 177us/step - loss: 3204.6621 - acc: 0.0000e+00
Epoch 2374/2500
216/216 [==============================] - 0s 48us/step - loss: 3204.4839 - acc: 0.0000e+00
Epoch 2375/2500
216/216 [==============================] - 0s 101us/step - loss: 3204.3027 - acc: 0.0000e+00
Epoch 2376/2500
216/216 [==============================] - 0s 137us/step - loss: 3204.1284 - acc: 0.0000e+00
Epoch 2377/2500
216/216 [==============================] - 0s 248us/step - loss: 3203.9583 - acc: 0.0000e+00
Epoch 2378/2500
216/216 [==============================] - 0s 196us/step - loss: 3203.7815 - acc: 0.0000e+00
Epoch 2379/2500
216/216

Epoch 2446/2500
216/216 [==============================] - 0s 86us/step - loss: 3192.8684 - acc: 0.0000e+00
Epoch 2447/2500
216/216 [==============================] - 0s 167us/step - loss: 3192.7268 - acc: 0.0000e+00
Epoch 2448/2500
216/216 [==============================] - 0s 95us/step - loss: 3192.5823 - acc: 0.0000e+00
Epoch 2449/2500
216/216 [==============================] - 0s 115us/step - loss: 3192.4421 - acc: 0.0000e+00
Epoch 2450/2500
216/216 [==============================] - 0s 55us/step - loss: 3192.3057 - acc: 0.0000e+00
Epoch 2451/2500
216/216 [==============================] - 0s 71us/step - loss: 3192.1646 - acc: 0.0000e+00
Epoch 2452/2500
216/216 [==============================] - 0s 91us/step - loss: 3192.0276 - acc: 0.0000e+00
Epoch 2453/2500
216/216 [==============================] - 0s 302us/step - loss: 3191.8923 - acc: 0.0000e+00
Epoch 2454/2500
216/216 [==============================] - 0s 165us/step - loss: 3191.7568 - acc: 0.0000e+00
Epoch 2455/2500
216/216 

In [341]:
combined_model.evaluate([name_bow_test, category_test] + [test_embed], labels_testOne, batch_size=512)

216/216 [==============================] - 1s 4ms/step


[3186.030029296875, 0.0]

In [342]:
# Generate predictions
predictions = combined_model.predict([name_bow_test, category_test] + [test_embed])

In [343]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 50
diff = 0
num_correct = 0

for i in range(num_predictions):
    val = predictions[i]
    print(name_test.iloc[i])
    print('Predicted Price: ', val[0], 'Actual Price: ', labels_testOne.iloc[i], '\n')
    diff += abs(val[0] - labels_testOne.iloc[i])
    if(abs(val[0] - labels_testOne.iloc[i]) <= 10):
        num_correct += 1

Amrut Peated Indian Single Malt Whiskey Cask Strength
Predicted Price:  81.061806 Actual Price:  85.0 

Talisker 1992 Single Malt Scotch
Predicted Price:  68.26414 Actual Price:  68.0 

Balcones True Blue Cask Strength Corn Whiskey
Predicted Price:  39.671185 Actual Price:  59.0 

Breckenridge Bourbon Whiskey
Predicted Price:  36.900692 Actual Price:  39.0 

Glenfiddich 18 Year Old Single Malt Scotch
Predicted Price:  69.57818 Actual Price:  79.0 

Greenore 15 Year Old Single Malt Irish Whiskey
Predicted Price:  79.65747 Actual Price:  100.0 

Amrut Indian Single Malt Whisky
Predicted Price:  66.803566 Actual Price:  36.0 

Isle of Jura 16 Year Old Single Malt Scotch Whisky
Predicted Price:  59.976585 Actual Price:  63.0 

Forty Creek Double Barrel Canadian Whiskey
Predicted Price:  54.024853 Actual Price:  60.0 

Johnnie Walker Gold Label Blended Scotch Whisky
Predicted Price:  70.73223 Actual Price:  73.0 

Tyrconnell Single Malt Irish Whiskey
Predicted Price:  46.048992 Actual Price

In [344]:
# Compare the average difference between actual price and the model's predicted price
print('Average prediction difference: ', diff / num_predictions)

# Print the accuracy
correct_percentage = (num_correct/num_predictions) * 100
print(correct_percentage, "%")

Average prediction difference:  28.520983276367186
40.0 %


In [345]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 50
diff = 0
num_correct = 0

for i in range(num_predictions):
    val = predictions[i]
    print(name_test.iloc[i])
    print('Predicted Rating: ', val[0], 'Actual Rating: ', labels_testTwo.iloc[i], '\n')
    diff += abs(val[0] - labels_testTwo.iloc[i])
    if(abs(val[0] - labels_testTwo.iloc[i]) <= 2):
        num_correct += 1

Amrut Peated Indian Single Malt Whiskey Cask Strength
Predicted Rating:  81.061806 Actual Rating:  80.0 

Talisker 1992 Single Malt Scotch
Predicted Rating:  68.26414 Actual Rating:  94.0 

Balcones True Blue Cask Strength Corn Whiskey
Predicted Rating:  39.671185 Actual Rating:  88.0 

Breckenridge Bourbon Whiskey
Predicted Rating:  36.900692 Actual Rating:  88.0 

Glenfiddich 18 Year Old Single Malt Scotch
Predicted Rating:  69.57818 Actual Rating:  90.0 

Greenore 15 Year Old Single Malt Irish Whiskey
Predicted Rating:  79.65747 Actual Rating:  88.0 

Amrut Indian Single Malt Whisky
Predicted Rating:  66.803566 Actual Rating:  76.0 

Isle of Jura 16 Year Old Single Malt Scotch Whisky
Predicted Rating:  59.976585 Actual Rating:  80.0 

Forty Creek Double Barrel Canadian Whiskey
Predicted Rating:  54.024853 Actual Rating:  88.0 

Johnnie Walker Gold Label Blended Scotch Whisky
Predicted Rating:  70.73223 Actual Rating:  78.0 

Tyrconnell Single Malt Irish Whiskey
Predicted Rating:  46

In [346]:
# Compare the average difference between actual rating and the model's predicted rating
print('Average prediction difference: ', diff / num_predictions)

# Print the accuracy
correct_percentage = (num_correct/num_predictions) * 100
print(correct_percentage, "%")

Average prediction difference:  31.270326995849608
8.0 %
